# 求导

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_grad_operation.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_grad_operation.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/intermediate/grad_operation.ipynb)

`mindspore.ops`模块提供的`GradOperation`接口可以生成网络模型的梯度。本文主要介绍如何使用`GradOperation`接口进行一阶、二阶求导，以及如何停止计算梯度，更多接口相关信息可参考[API文档](https://mindspore.cn/docs/api/zh-CN/master/api_python/ops/mindspore.ops.GradOperation.html#mindspore.ops.GradOperation)。

## 一阶求导

计算一阶导数方法：`mindspore.ops.GradOperation (get_all=False, get_by_list=False, sens_param=False)`，其中参数使用方式为：

- `get_all`：为`False`时，只会对第一个输入求导；为`True`时，会对所有输入求导。
- `get_by_list：`为`False`时，不会对权重求导；为`True`时，会对权重求导。
- `sens_param`：对网络的输出值做缩放以改变最终梯度，故其维度与输出维度保持一致；

下面我们先使用[MatMul](https://mindspore.cn/docs/api/zh-CN/master/api_python/ops/mindspore.ops.MatMul.html#mindspore.ops.MatMul)算子构建自定义网络模型`Net`，再对其进行一阶求导，通过这样一个例子对`GradOperation`接口的使用方式做简单介绍。首先我们要定义网络模型`Net`、输入`x`和输入`y`：

In [1]:
import numpy as np
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Tensor
from mindspore import ParameterTuple, Parameter
from mindspore import dtype as mstype

# 定义输出x和y
x = Tensor([[0.8, 0.6, 0.2], [1.8, 1.3, 1.1]], dtype=mstype.float32)
y = Tensor([[0.11, 3.3, 1.1], [1.1, 0.2, 1.4], [1.1, 2.2, 0.3]], dtype=mstype.float32)

# 定义矩阵相乘网络Net
class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        # 矩阵相乘
        self.matmul = ops.MatMul()
        # 定义权重
        self.z = Parameter(Tensor(np.array([1.0], np.float32)), name='z')
    def construct(self, x, y):
        x = x * self.z
        out = self.matmul(x, y)
        return out

### 对输入进行求导

对输入值进行求导，代码如下：

In [2]:
# 定义网络输入的一阶求导
class GradNetWrtX(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtX, self).__init__()
        self.net = net
        self.grad_op = ops.GradOperation()
    def construct(self, x, y):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x, y)

output = GradNetWrtX(Net())(x, y)
print(output)

[[4.51      2.7       3.6000001]
 [4.51      2.7       3.6000001]]


接下来我们对上面的结果做一个解释。为便于分析，我们把上面的输入`x`、`y`以及权重`z`表示成如下形式:

```text
x = Tensor([[x1, x2, x3], [x4, x5, x6]])
y = Tensor([[y1, y2, y3], [y4, y5, y6], [y7, y8, y9]])
z = Tensor([z])
```

根据MatMul算子定义可得前向结果：

$output = [[(x_1 \cdot y_1 + x_2 \cdot y_4 + x_3 \cdot y_7) \cdot z, (x_1 \cdot y_2 + x_2 \cdot y_5 + x_3 \cdot y_8) \cdot z, (x_1 \cdot y_3 + x_2 \cdot y_6 + x_3 \cdot y_9) \cdot z],
[(x_4 \cdot y_1 + x_5 \cdot y_4 + x_6 \cdot y_7) \cdot z, (x_4 \cdot y_2 + x_5 \cdot y_5 + x_6 \cdot y_8) \cdot z, (x_4 \cdot y_3 + x_5 \cdot y_6 + x_6 \cdot y_9) \cdot z]]$

梯度计算时由于MindSpore采用的是Reverse自动微分机制，会对输出结果求和后再对输入`x`求导：

1. 求和公式：

$\sum{output} = [(x_1 \cdot y_1 + x_2 \cdot y_4 + x_3 \cdot y_7) + (x_1 \cdot y_2 + x_2 \cdot y_5 + x_3 \cdot y_8) + (x_1 \cdot y_3 + x_2 \cdot y_6 + x_3 \cdot y_9) +$

$(x_4 \cdot y_1 + x_5 \cdot y_4 + x_6 \cdot y_7) + (x_4 \cdot y_2 + x_5 \cdot y_5 + x_6 \cdot y_8) + (x_4 \cdot y_3 + x_5 \cdot y_6 + x_6 \cdot y_9)] \cdot z$

2. 求导公式：

$\frac{\mathrm{d}(\sum{output})}{\mathrm{d}x} = [[(y_1 + y_2 + y_3) \cdot z，(y_4 + y_5 + y_6) \cdot z，(y_7 + y_8 + y_9) \cdot z]，[(y_1 + y_2 + y_3) \cdot z，(y_4 + y_5 + y_6) \cdot z，(y_7 + y_8 + y_9) \cdot z]]$

3. 计算结果：

$\frac{\mathrm{d}(\sum{output})}{\mathrm{d}x} = [[4.5099998 \quad 2.7 \quad 3.6000001] [4.5099998 \quad 2.7 \quad 3.6000001]]$

> 若考虑对`x`、`y`输入求导，只需在`GradNetWrtX`中设置`self.grad_op = GradOperation(get_all=True)`。

### 对权重进行求导

对权重进行求导，示例代码如下：

In [3]:
# 定义网络权重的一阶求导
class GradNetWrtZ(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtZ, self).__init__()
        self.net = net
        self.params = ParameterTuple(net.trainable_params())
        self.grad_op = ops.GradOperation(get_by_list=True)
    def construct(self, x, y):
        gradient_function = self.grad_op(self.net, self.params)
        return gradient_function(x, y)

output = GradNetWrtZ(Net())(x, y)
print(output[0])

[21.536001]


下面我们通过公式对上面的结果做一个解释。对权重的求导公式为：

$\frac{\mathrm{d}(\sum{output})}{\mathrm{d}z} = (x_1 \cdot y_1 + x_2 \cdot y_4 + x_3 \cdot y_7) + (x_1 \cdot y_2 + x_2 \cdot y_5 + x_3 \cdot y_8) + (x_1 \cdot y_3 + x_2 \cdot y_6 + x_3 \cdot y_9) + $

$(x_4 \cdot y_1 + x_5 \cdot y_4 + x_6 \cdot y_7) + (x_4 \cdot y_2 + x_5 \cdot y_5 + x_6 \cdot y_8) + (x_4 \cdot y_3 + x_5 \cdot y_6 + x_6 \cdot y_9)$

计算结果：

$\frac{\mathrm{d}(\sum{output})}{\mathrm{d}z} = [2.15359993e+01]$

### 梯度值缩放

可以通过`sens_param`参数控制梯度值的缩放：

In [4]:
# 定义网络的一阶求导,控制梯度值缩放
class GradNetWrtN(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtN, self).__init__()
        self.net = net
        self.grad_op = ops.GradOperation(sens_param=True)
        # 定义梯度值缩放
        self.grad_wrt_output = Tensor([[0.1, 0.6, 0.2], [0.8, 1.3, 1.1]], dtype=mstype.float32)
    def construct(self, x, y):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x, y, self.grad_wrt_output)
output = GradNetWrtN(Net())(x, y)
print(output)

[[2.211     0.51      1.49     ]
 [5.5880003 2.68      4.0699997]]


为了方便对上面的结果进行解释，我们把`self.grad_wrt_output`记作如下形式：

```text
self.grad_wrt_output = Tensor([[s1, s2, s3], [s4, s5, s6]])
```

缩放后的输出值为原输出值与`self.grad_wrt_output`对应元素的乘积，公式为：

$output = [[(x_1 \cdot y_1 + x_2 \cdot y_4 + x_3 \cdot y_7) \cdot z \cdot s_1，(x_1 \cdot y_2 + x_2 \cdot y_5 + x_3 \cdot y_8) \cdot z \cdot s_2，(x_1 \cdot y_3 + x_2 \cdot y_6 + x_3 \cdot y_9) \cdot z \cdot s_3]，$

$[(x_4 \cdot y_1 + x_5 \cdot y_4 + x_6 \cdot y_7) \cdot z \cdot s_4，(x_4 \cdot y_2 + x_5 \cdot y_5 + x_6 \cdot y_8) \cdot z \cdot s_5，(x_4 \cdot y_3 + x_5 \cdot y_6 + x_6 \cdot y_9) \cdot z \cdot s_6]]$

求导公式变为输出值总和对`x`的每个元素求导：

$\frac{\mathrm{d}(\sum{output})}{\mathrm{d}x} = [[(s_1 \cdot y_1 + s_2 \cdot y_2 + s_3 \cdot y_3) \cdot z，(s_1 \cdot y_4 + s_2 \cdot y_5 + s_3 \cdot y_6) \cdot z，(s_1 \cdot y_7 + s_2 \cdot y_8 + s_3 \cdot y_9) \cdot z]，$

$[(s_4 \cdot y_1 + s_5 \cdot y_2 + s_6 \cdot y_3) \cdot z，(s_4 \cdot y_4 + s_5 \cdot y_5 + s_6 \cdot y_6) \cdot z，(s_4 \cdot y_7 + s_5 \cdot y_8 + s_6 \cdot y_9) \cdot z]]$

计算结果：

$\frac{\mathrm{d}(\sum{output})}{\mathrm{d}z} = [[2.211 \quad 0.51 \quad 1.49][5.5880003 \quad 2.68 \quad 4.0699997]]$

## 停止计算梯度

我们可以使用`stop_gradient`来停止计算指定算子的梯度，从而消除该算子对梯度的影响。在上面一阶求导使用的矩阵相乘网络模型的基础上，我们再增加一个算子`out2`并禁止计算其梯度，得到自定义网络`Net2`，然后看一下对输入的求导结果情况。示例代码如下：

In [5]:
# 导入stop_gradient模块
from mindspore.ops import stop_gradient

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.matmul = ops.MatMul()

    def construct(self, x, y):
        out1 = self.matmul(x, y)
        out2 = self.matmul(x, y)
        # 停止计算out2算子的梯度
        out2 = stop_gradient(out2)
        out = out1 + out2
        return out

class GradNetWrtX(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtX, self).__init__()
        self.net = net
        self.grad_op = ops.GradOperation()

    def construct(self, x, y):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x, y)

output = GradNetWrtX(Net())(x, y)
print(output)

[[4.51      2.7       3.6000001]
 [4.51      2.7       3.6000001]]


从上面的打印可以看出，由于对`out2`设置了`stop_gradient`, 所以`out2`没有对梯度计算有任何的贡献，其输出结果与未加`out2`算子时一致。下面我们删除`out2 = stop_gradient(out2)`，再来看一下输出结果。示例代码为：

In [6]:
class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.matmul = ops.MatMul()

    def construct(self, x, y):
        out1 = self.matmul(x, y)
        out2 = self.matmul(x, y)
        # out2 = stop_gradient(out2)
        out = out1 + out2
        return out

class GradNetWrtX(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtX, self).__init__()
        self.net = net
        self.grad_op = ops.GradOperation()

    def construct(self, x, y):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x, y)

output = GradNetWrtX(Net())(x, y)
print(output)

[[9.02      5.4       7.2000003]
 [9.02      5.4       7.2000003]]


从上面的打印结果可以看出，在我们把`out2`算子的梯度也计算进去之后，由于`out2`和`out1`算子完全相同，因此它们产生的梯度也完全相同，所以我们可以看到，结果中每一项的值都变为了原来的两倍。

## 高阶求导

高阶微分在AI支持科学计算、二阶优化等领域均有应用。如分子动力学模拟中，利用神经网络训练势能时，损失函数中需计算神经网络输出对输入的导数，则反向传播便存在损失函数对输入、权重的二阶交叉导数；此外，AI求解微分方程（如PINNs方法）还会存在输出对输入的二阶导数。又如二阶优化中，为了能够让神经网络快速收敛，牛顿法等需计算损失函数对权重的二阶导数。以下将主要介绍MindSpore图模式下的高阶导数。

MindSpore可通过多次求导的方式支持高阶导数，下面通过几类例子展开阐述。

### 单输入单输出高阶导数

例如Sin算子，其二阶导数（-Sin）实现如下：

In [7]:
import numpy as np
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Tensor

# 定义基于Sin算子的网络模型
class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.sin = ops.Sin()
    def construct(self, x):
        out = self.sin(x)
        return out
# 一阶求导
class Grad(nn.Cell):
    def __init__(self, network):
        super(Grad, self).__init__()
        self.grad = ops.GradOperation()
        self.network = network
    def construct(self, x):
        gout = self.grad(self.network)(x)
        return gout

# 二阶求导
class GradSec(nn.Cell):
    def __init__(self, network):
        super(GradSec, self).__init__()
        self.grad = ops.GradOperation()
        self.network = network
    def construct(self, x):
        gout = self.grad(self.network)(x)
        return gout

net = Net()
firstgrad = Grad(net)
secondgrad = GradSec(firstgrad)
x_train = Tensor(np.array([3.1415926]))
output = secondgrad(x_train)
# 打印结果
print(output)

[-5.35897932e-08]


从上面的打印结果可以看出，`-sin(3.1415926)`的值接近于`0`。